In [112]:
%cd '/content/drive/MyDrive/Colab Notebooks/courses/stepik/m5. tasks/w4. recommender'
!ls

/content/drive/MyDrive/Colab Notebooks/courses/stepik/m5. tasks/w4. recommender
'4.3 recommender.ipynb'
 coursera_sessions_test.txt
 coursera_sessions_train.txt
'DataAnalisysMipt\Results\Test_Purchases frequency model.txt'
'DataAnalisysMipt\Results\Test_View frequency model.txt'
'DataAnalisysMipt\Results\Train_Purchases frequency model.txt'
'DataAnalisysMipt\Results\Train_View frequency model.txt'
 github.ipynb


In [113]:
import pandas as pd
from collections import defaultdict

In [119]:
def read_dataset(fname):
    data = pd.read_csv(fname, sep=';', names=['views', 'purchases'])
    for c in data.columns:
        data[c] = data[c].apply(
            lambda x: [int(v) for v in x.split(',')] if pd.notna(x) else x)
    return data

train = read_dataset('coursera_sessions_train.txt')
test = read_dataset('coursera_sessions_test.txt')


In [115]:
def get_counts(x):
    x = x.dropna()
    counter = defaultdict(int)
    for lst in x:
        for val in lst:
            counter[val] += 1
    return counter


def remove_dups(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]


def sort_by_counts(views, counts):
    return sorted(views, key=lambda x: counts[x], reverse=True)

In [120]:
def get_precision(row, k):
    recommended = row['views'][:k]
    purchased = row['purchases']
    precision = [item for item in purchased if item in recommended]
    return len(precision) / k#len(recommended[:k])


def get_recall(row, k):
    recommended = row['views'][:k]
    purchased = row['purchases']
    recall = [item for item in recommended if item in purchased]
    return len(recall) / len(purchased)


def get_metrics(df, counts):
    metrics = df[df['purchases'].notna()].copy()
    metrics['views'] = metrics['views'].apply(remove_dups)
    metrics['views'] = metrics['views'].apply(sort_by_counts, counts=counts)
    metrics = [[round(metrics.apply(f, k=k, axis=1).mean(), 4) 
        for f in (get_recall, get_precision)] for k in [1, 5]]
    return sum(metrics, [])

view_counts = get_counts(train['views'])
purchase_counts = get_counts(train['purchases'])
print(get_metrics(train, view_counts))
print(get_metrics(train, purchase_counts))
print(get_metrics(test, view_counts))
print(get_metrics(test, purchase_counts))

[0.4426, 0.5122, 0.8247, 0.2125]
[0.6884, 0.8038, 0.9263, 0.2525]
[0.4173, 0.4813, 0.8, 0.2038]
[0.4606, 0.5277, 0.8202, 0.2101]


In [117]:
[0.4426, 0.5122, 0.8247, 0.2125]

[0.4426, 0.5122, 0.8247, 0.2125]